In [1]:
import pandas as pd

## manually generate state code lists for regions
North: JK, ~~`[Ladakh]`~~, PN, HP, HR, UK, Delhi, Chandigarh 

West: RJ, GJ, MH, Goa, Dadra & Nagar Haveli, Daman & Diu  

Central: MP, UP, CG  

East: BH, WB, OR, JH  

South: KT, ~~`[TG]`~~, AP, TN, KL, Lakshadweep, Puducherry 

North-East: AS, SK, MG, TR, AR, MN, NG, MZ, Andaman & NicobarFor

**NOTE:** Since TG and Ladakh are not in the data files I have not included them 

In [2]:
North=[1,2,3,4,5,6,7]
West=[8,24,25,26,27,30]
Central=[9,22,23]
East=[10,19,20,21]
South=[28,29,31,32,33,34]
North_East=[11,12,13,14,15,16,17,18,35]
REGIONS=[North,West,Central,East,South,North_East]
REGION_NAMES=['North','West','Central','East','South','North_East']

## part-a

In [3]:
c16=pd.read_excel('datasets/C-16.xlsx',engine='openpyxl',skiprows=6,header=None)

In [4]:
def mother_toung(region,region_name):
    overallDF=pd.read_excel('datasets/C-16.xlsx',engine='openpyxl',skiprows=6,header=None)
    true_false_list=[]

    for code in c16.iloc[:,1].values:
        if code in region:
            true_false_list.append(True)
        else:
            true_false_list.append(False)
        
    df=overallDF[true_false_list]
    # print(df)
    mother_toung=[]
    langs=[]
    for i,lang in enumerate(df.iloc[:,6].values):

        if lang.isupper():
            item={
                'lang':lang[2:].strip(' '), # remove start and end whitspaces
                'count':df.iloc[i,7]
            }
            mother_toung.append(item)

    foo_df=pd.DataFrame(mother_toung) 
    foo_df=foo_df.groupby(['lang'],sort=False).sum()
    foo_df.sort_values('count',ascending=False,inplace=True)
    
    lang_list=list(foo_df.head(3).index)

    item={
        'region':region_name,
        'language-1':lang_list[0],
        'language-2':lang_list[1],
        'language-3':lang_list[2],
    }


    return item

In [5]:
region_list=[]
for region,region_name in zip(REGIONS,REGION_NAMES):
    region_list.append(mother_toung(region,region_name))

In [6]:
part_a_df=pd.DataFrame(region_list)

In [7]:
part_a_df.to_csv('outputs/region-india-a.csv',index=False)

## part-b

In [8]:
def getTopLangs(region,region_name):
    overallDF=pd.DataFrame()
    for code in region:
        df=pd.read_excel(f'datasets/C-17/{str(code).zfill(2)}.xlsx',skiprows=6,header=None,engine='openpyxl')
        overallDF=pd.concat([overallDF,df],ignore_index=True)
    overallDF.fillna(value=0,inplace=True)

    # cover all languages in 3 cols
    possibleLangs=overallDF.iloc[:,3].to_list()
    possibleLangs.extend(overallDF.iloc[:,8].to_list())
    possibleLangs.extend(overallDF.iloc[:,13].to_list())

    availableLangs=[]
    for langs in possibleLangs:
        if (langs!=0) and (langs not in availableLangs):
            availableLangs.append(langs)
    # print('total languages spoken in the region are >>> ',len(availableLangs))
    langCounts={}
    for lang in availableLangs:
        langCounts[lang]=0
    for lang in availableLangs:
        for i,data in enumerate(overallDF.iloc[:,3].to_list()):
            if data==lang:
                langCounts[lang]=langCounts[lang]+overallDF.iloc[i,4]
        for i,data in enumerate(overallDF.iloc[:,8].to_list()):
            if data==lang:
                langCounts[lang]=langCounts[lang]+overallDF.iloc[i,9]
        for i,data in enumerate(overallDF.iloc[:,13].to_list()):
            if data==lang:
                langCounts[lang]=langCounts[lang]+overallDF.iloc[i,14]
    # store in a df and sort
    langCountsDF=pd.DataFrame({'name':list(langCounts.keys()),'count':list(langCounts.values())})
    langCountsDF.sort_values('count',axis=0,ascending=False,inplace=True)

    lang_list=langCountsDF.head(3)['name'].to_list()

    item={
        'region':region_name,
        'language-1':lang_list[0],
        'language-2':lang_list[1],
        'language-3':lang_list[2],
    }

    return item

In [9]:
region_list=[]
for region,region_name in zip(REGIONS,REGION_NAMES):
    region_list.append(getTopLangs(region,region_name))

In [10]:
part_b_df=pd.DataFrame(region_list)

In [11]:
part_b_df.to_csv('outputs/region-india-b.csv',index=False)

- it makes sense that both occurances of English is being replaced by local regional language, because people normally does not use English in normal daily activites; rather they use their local mother toung more in daily activities
    - In north: English --> Kashmiri
    - In central: Urdu becomes 2nd and 3rd is Bhili